## 압축 풀기(한번만)

In [ ]:
# input data 압 축 풀 기 한 번 만 ! 
%cd /content/
import os
import zipfile
import shutil
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import zipfile

from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/data/'
os.makedirs('./data/', exist_ok=True)

output_unzip = zipfile.ZipFile("/content/drive/MyDrive/project/BreastCancer/input/open.zip", "r")  # "r": read 모드
output_unzip.extractall(file_path)
output_unzip.close()

os.makedirs('./data/train_imgs/0', exist_ok=True)
os.makedirs('./data/train_imgs/1', exist_ok=True)
os.makedirs('./data/test_imgs', exist_ok=True)

/content
Mounted at /content/drive


## 데이터셋(한번만)

In [ ]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders

In [ ]:
train_df = pd.read_csv('./data/train.csv')[['ID', 'N_category']]

In [ ]:
# 데이터 라벨 스플릿
for i in tqdm(os.listdir('./data/train_imgs')):
  if len(i) == 1:
    continue
  j = i.replace('.png', '').split('__')[0]
  k = train_df[train_df['ID']==j].values[0][1]
  shutil.move(f'./data/train_imgs/{i}', f'./data/train_imgs/{k}')

  0%|          | 0/1002 [00:00<?, ?it/s]

In [ ]:
# # 데이터 라벨 스플릿
# for i, j in tqdm(train_df.values):
#   shutil.move(f'./data/train_imgs/{i}.png', f'./data/train_imgs/{j}')

In [ ]:
# 데이터 폴더스플릿
splitfolders.ratio('./data/train_imgs', output="./data/train_imgs", seed=88, ratio=(.8, .2))
for i in range(2):
  shutil.rmtree(f'./data/train_imgs/{i}')

Copying files: 1000 files [01:10, 14.14 files/s]


In [ ]:
# 데이터셋 구축
for i, j in zip(['training_set', 'val_set', 'test_set'], ['train', 'val', 'NONE']):
  os.makedirs(f'./data/dataset/{i}/{i}', exist_ok=True)
  if i == 'test_set':
    break
# for j in ['train', 'val']:
  f = open(f'./data/dataset/{j}.txt', 'w')
  for k in range(2):
    for l in os.listdir(f'./data/train_imgs/{j}/{k}'):
      f.write(f'{k}/{l} {k}\n')
    shutil.move(f'./data/train_imgs/{j}/{k}', f'./data/dataset/{i}/{i}')
  f.close()


f = open('./data/dataset/test.txt', 'w')
for i in os.listdir('./data/test_imgs'):
  shutil.move(f'./data/test_imgs/{i}', './data/dataset/test_set/test_set')
  f.write(f'{i}\n')
f.close()


f = open('./data/dataset/classes.txt', 'w') # txt파일 경로
for i in range(2):
  f.write(f'{i}\n')
f.close()

shutil.rmtree('./data/train_imgs')
shutil.rmtree('./data/test_imgs')

In [ ]:
len(os.listdir('./data/dataset/training_set/training_set/0')), len(os.listdir('./data/dataset/training_set/training_set/1'))

(388, 411)

## 설치

In [ ]:
!nvidia-smi

Sun Nov 20 13:00:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvcc -V
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# Install mmcv
!pip install mmcv -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.9.0/index.html
     |████████████████████████████████| 593 kB 32.6 MB/s 
     |████████████████████████████████| 190 kB 73.7 MB/s 
  Created wheel for mmcv: filename=mmcv-1.7.0-py2.py3-none-any.whl size=914087 sha256=f717e859e8e5fe273faa29268be45bbfd15a0116f6cd94140a13e96ff1648225
  Stored in directory: /root/.cache/pip/wheels/09/fb/52/8c066c45e9be328685e3094b76900832a850a0ed614ed2398d
Successfully built mmcv


In [ ]:
!git clone https://github.com/open-mmlab/mmclassification.git
%cd mmclassification/
!pip install -e . 

Cloning into 'mmclassification'...
remote: Enumerating objects: 11018, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 11018 (delta 50), reused 67 (delta 22), pack-reused 10877
Receiving objects: 100% (11018/11018), 10.73 MiB | 14.91 MiB/s, done.
Resolving deltas: 100% (7625/7625), done.
/content/mmclassification
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmclassification
  Running setup.py develop for mmcls


In [ ]:
!pip install mmcls
!pip install mmcv-full

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 593 kB 32.2 MB/s 
ERROR: Operation cancelled by user


# 3.res2net50-w14-s8_8xb32_in1k

In [ ]:
# res2net50-w14-s8_8xb32_in1k
config_file = r'/content/mmclassification/configs/res2net/res2net50-w14-s8_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth'

## model 1 : 1.00E-04 = 0.6500

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='res2net50-w14-s8_8xb32_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################


normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

### Train

In [ ]:
%cd /content/

In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

In [ ]:
# 출력
display(df_submission)

In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

## model 2 : 1.00E-03 = 0.6325146771

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='res2net50-w14-s8_8xb32_in1k'
VERSION = 2
LEARNING_RATE = 1e-3
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=8,
        base_width=14,
        deep_stem=False,
        avg_down=False,
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type=

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 03:02:59,580 - mmcv - INFO - initialize Res2Net with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth', 'prefix': 'backbone'}
2022-11-20 03:02:59,582 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth" to /root/.cache/torch/hub/checkpoints/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth


  0%|          | 0.00/96.0M [00:00<?, ?B/s]

2022-11-20 03:03:09,641 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 03:03:09,643 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 03:03:09,645 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 03:03:09,646 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 03:03:09,648 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([112, 64, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassificat

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.5 task/s, elapsed: 44s, ETA:     0s

...

2022-11-20 04:41:00,275 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 69.6517
2022-11-20 04:41:35,905 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-31, eta: 0:04:57, time: 3.562, data_time: 2.345, memory: 10568, loss: 0.5290
2022-11-20 04:42:07,359 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-31, eta: 0:04:24, time: 3.145, data_time: 1.929, memory: 10568, loss: 0.5213
2022-11-20 04:42:39,301 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-31, eta: 0:03:51, time: 3.194, data_time: 1.972, memory: 10568, loss: 0.5506
2022-11-20 04:43:11,659 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-31, eta: 0:03:18, time: 3.236, data_time: 2.000, memory: 10568, loss: 0.5546
2022-11-20 04:43:41,347 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-31, eta: 0:02:44, time: 2.969, data_time: 1.749, memory: 10568, loss: 0.5851
2022-11-20 04:43:41,351 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-20 04:44:25,320 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 67.6617
2022-11-20 04:45:00,647 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-32, eta: 0:02:12, time: 3.532, data_time: 2.319, memory: 10568, loss: 0.6138
2022-11-20 04:45:32,504 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-32, eta: 0:01:38, time: 3.186, data_time: 1.964, memory: 10568, loss: 0.5394
2022-11-20 04:46:04,025 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-32, eta: 0:01:05, time: 3.152, data_time: 1.919, memory: 10568, loss: 0.5517
2022-11-20 04:46:35,056 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-32, eta: 0:00:32, time: 3.103, data_time: 1.874, memory: 10568, loss: 0.5424
2022-11-20 04:47:05,835 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-32, eta: 0:00:00, time: 3.078, data_time: 1.854, memory: 10568, loss: 0.5174
2022-11-20 04:47:05,839 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-20 04:47:49,852 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 69.1542


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.821778
1,BC_01_0220,0,0.729722
2,BC_01_0233,0,0.661563
3,BC_01_0258,1,0.536290
4,BC_01_0260,1,0.575515
...,...,...,...
245,BC_01_3328,0,0.941761
246,BC_01_3404,1,0.565381
247,BC_01_3418,0,0.744697
248,BC_01_3438,0,0.606774


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

# 4.resnet50_8xb32-fp16_in1k

In [ ]:
# resnet50_8xb32-fp16_in1k
config_file = r'/content/mmclassification/configs/resnet/resnet50_8xb32-fp16_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth'

## model 1 : 1.00E-04 = 0.6371

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnet50_8xb32-fp16_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28,

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-19 13:34:01,595 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth', 'prefix': 'backbone'}
2022-11-19 13:34:01,596 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth" to /root/.cache/torch/hub/checkpoints/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth


  0%|          | 0.00/49.1M [00:00<?, ?B/s]

2022-11-19 13:34:05,166 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-19 13:34:05,169 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth 
 
2022-11-19 13:34:05,170 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth 
 
2022-11-19 13:34:05,172 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batch256_fp16_imagenet_20210320-b3964210.pth 
 
2022-11-19 13:34:05,174 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/fp16/resnet50_batc

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

...

2022-11-19 15:10:18,781 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 68.1592
2022-11-19 15:10:53,448 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-32, eta: 0:04:52, time: 3.466, data_time: 3.112, memory: 10384, loss: 0.3218
2022-11-19 15:11:25,348 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-32, eta: 0:04:19, time: 3.190, data_time: 2.839, memory: 10384, loss: 0.4214
2022-11-19 15:11:56,581 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-32, eta: 0:03:47, time: 3.123, data_time: 2.774, memory: 10384, loss: 0.3900
2022-11-19 15:12:28,476 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-32, eta: 0:03:14, time: 3.190, data_time: 2.818, memory: 10384, loss: 0.3574
2022-11-19 15:12:58,985 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-32, eta: 0:02:42, time: 3.051, data_time: 2.696, memory: 10384, loss: 0.3893
2022-11-19 15:12:58,990 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.8 task/s, elapsed: 42s, ETA:     0s

2022-11-19 15:13:42,446 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 67.6617
2022-11-19 15:14:17,131 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-33, eta: 0:02:09, time: 3.468, data_time: 3.120, memory: 10384, loss: 0.3930
2022-11-19 15:14:48,208 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-33, eta: 0:01:37, time: 3.108, data_time: 2.751, memory: 10384, loss: 0.3518
2022-11-19 15:15:19,848 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-33, eta: 0:01:04, time: 3.164, data_time: 2.810, memory: 10384, loss: 0.4126
2022-11-19 15:15:51,192 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-33, eta: 0:00:32, time: 3.134, data_time: 2.781, memory: 10384, loss: 0.3361
2022-11-19 15:16:21,832 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-33, eta: 0:00:00, time: 3.064, data_time: 2.712, memory: 10384, loss: 0.3455
2022-11-19 15:16:21,836 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 42s, ETA:     0s

2022-11-19 15:17:05,332 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 66.6667


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.912826
1,BC_01_0220,0,0.908537
2,BC_01_0233,0,0.928991
3,BC_01_0258,0,0.692226
4,BC_01_0260,1,0.670743
...,...,...,...
245,BC_01_3328,1,0.643637
246,BC_01_3404,1,0.549958
247,BC_01_3418,0,0.853608
248,BC_01_3438,1,0.568549


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

## model 2 : 1.00E-03 = 0.6325146771

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnet50_8xb32-fp16_in1k'
VERSION = 2
LEARNING_RATE = 1e-3
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=8,
        base_width=14,
        deep_stem=False,
        avg_down=False,
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type=

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 04:50:00,831 - mmcv - INFO - initialize Res2Net with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth', 'prefix': 'backbone'}
2022-11-20 04:50:00,832 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth
2022-11-20 04:50:00,973 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 04:50:00,975 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 04:50:00,976 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdpar

load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth


2022-11-20 04:50:01,034 - mmcv - INFO - 
backbone.layer1.1.conv3.weight - torch.Size([256, 112, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 04:50:01,035 - mmcv - INFO - 
backbone.layer1.1.bn3.weight - torch.Size([256]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 04:50:01,037 - mmcv - INFO - 
backbone.layer1.1.bn3.bias - torch.Size([256]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 04:50:01,043 - mmcv - INFO - 
backbone.layer1.1.convs.0.weight - torch.Size([14, 14, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/res2net/res2net50-w14-s8_3rdparty_8xb32_in1k_20210927-bc967bf1.pth 
 
2022-11-20 04:50:01,

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

2022-11-20 04:53:34,396 - mmcls - INFO - Epoch(val) [1][13]	accuracy_top-1: 53.2338
2022-11-20 04:54:11,265 - mmcls - INFO - Epoch [2][10/50]	lr: 1.000e-04, eta: 1:21:32, time: 3.686, data_time: 2.474, memory: 10569, loss: 0.5962
2022-11-20 04:54:43,282 - mmcls - INFO - Epoch [2][20/50]	lr: 1.000e-04, eta: 1:20:18, time: 3.202, data_time: 1.987, memory: 10569, loss: 0.6150
2022-11-20 04:55:15,631 - mmcls - INFO - Epoch [2][30/50]	lr: 1.000e-04, eta: 1:19:21, time: 3.235, data_time: 2.014, memory: 10569, loss: 0.5418
2022-11-20 04:55:47,664 - mmcls - INFO - Epoch [2][40/50]	lr: 1.000e-04, eta: 1:18:24, time: 3.203, data_time: 1.984, memory: 10569, loss: 0.5599
2022-11-20 04:56:20,436 - mmcls - INFO - Epoch [2][50/50]	lr: 1.000e-04, eta: 1:17:42, time: 3.277, data_time: 2.068, memory: 10569, loss: 0.6280
2022-11-20 04:56:20,440 - mmcls - INFO - Saving checkpoint at 2 epochs


...

2022-11-20 06:27:12,756 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 69.6517
2022-11-20 06:27:48,504 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-31, eta: 0:04:55, time: 3.574, data_time: 2.360, memory: 10569, loss: 0.5290
2022-11-20 06:28:20,123 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-31, eta: 0:04:22, time: 3.162, data_time: 1.940, memory: 10569, loss: 0.5213
2022-11-20 06:28:52,093 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-31, eta: 0:03:49, time: 3.197, data_time: 1.973, memory: 10569, loss: 0.5506
2022-11-20 06:29:24,547 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-31, eta: 0:03:17, time: 3.245, data_time: 2.010, memory: 10569, loss: 0.5546
2022-11-20 06:29:54,429 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-31, eta: 0:02:44, time: 2.988, data_time: 1.767, memory: 10569, loss: 0.5851
2022-11-20 06:29:54,433 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 42s, ETA:     0s

2022-11-20 06:30:38,122 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 67.6617
2022-11-20 06:31:13,805 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-32, eta: 0:02:11, time: 3.568, data_time: 2.352, memory: 10569, loss: 0.6138
2022-11-20 06:31:46,006 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-32, eta: 0:01:38, time: 3.220, data_time: 1.997, memory: 10569, loss: 0.5394
2022-11-20 06:32:17,221 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-32, eta: 0:01:05, time: 3.121, data_time: 1.893, memory: 10569, loss: 0.5517
2022-11-20 06:32:48,438 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-32, eta: 0:00:32, time: 3.122, data_time: 1.897, memory: 10569, loss: 0.5424
2022-11-20 06:33:20,301 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-32, eta: 0:00:00, time: 3.186, data_time: 1.966, memory: 10569, loss: 0.5174
2022-11-20 06:33:20,306 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 42s, ETA:     0s

2022-11-20 06:34:03,885 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 69.1542


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.821778
1,BC_01_0220,0,0.729722
2,BC_01_0233,0,0.661563
3,BC_01_0258,1,0.536290
4,BC_01_0260,1,0.575515
...,...,...,...
245,BC_01_3328,0,0.941761
246,BC_01_3404,1,0.565381
247,BC_01_3418,0,0.744697
248,BC_01_3438,0,0.606774


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

# 5.resnet18_8xb32_in1k

In [ ]:
# resnet18_8xb32_in1k
config_file = r'/content/mmclassification/configs/resnet/resnet18_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth'

## model 1 : 1e-4 = 0.6425145313

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnet18_8xb32_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNet',
        depth=18,
        num_stages=4,
        out_indices=(3, ),
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=512,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
 

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 06:35:56,599 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth', 'prefix': 'backbone'}
2022-11-20 06:35:56,600 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth


2022-11-20 06:35:56,870 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 06:35:56,873 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth 
 
2022-11-20 06:35:56,874 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth 
 
2022-11-20 06:35:56,875 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth 
 
2022-11-20 06:35:56,877 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([64, 64, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnet/resnet18_8xb32_in1k_20210831-fbbb1da6.pth

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-20 06:39:24,484 - mmcls - INFO - Epoch(val) [1][13]	accuracy_top-1: 63.6816
2022-11-20 06:40:00,106 - mmcls - INFO - Epoch [2][10/50]	lr: 1.000e-05, eta: 1:19:09, time: 3.541, data_time: 3.306, memory: 10569, loss: 0.4939
2022-11-20 06:40:33,278 - mmcls - INFO - Epoch [2][20/50]	lr: 1.000e-05, eta: 1:18:40, time: 3.317, data_time: 3.075, memory: 10569, loss: 0.5312
2022-11-20 06:41:04,026 - mmcls - INFO - Epoch [2][30/50]	lr: 1.000e-05, eta: 1:17:27, time: 3.075, data_time: 2.827, memory: 10569, loss: 0.4774
2022-11-20 06:41:36,146 - mmcls - INFO - Epoch [2][40/50]	lr: 1.000e-05, eta: 1:16:45, time: 3.212, data_time: 2.965, memory: 10569, loss: 0.5085
2022-11-20 06:42:07,204 - mmcls - INFO - Epoch [2][50/50]	lr: 1.000e-05, eta: 1:15:50, time: 3.107, data_time: 2.870, memory: 10569, loss: 0.5475
2022-11-20 06:42:07,208 - mmcls - INFO - Saving checkpoint at 2 epochs


...

2022-11-20 08:11:35,252 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 68.6567
2022-11-20 08:12:09,197 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-32, eta: 0:04:49, time: 3.374, data_time: 3.140, memory: 10569, loss: 0.4072
2022-11-20 08:12:40,115 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-32, eta: 0:04:17, time: 3.091, data_time: 2.933, memory: 10569, loss: 0.4597
2022-11-20 08:13:11,396 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-32, eta: 0:03:44, time: 3.128, data_time: 3.004, memory: 10569, loss: 0.5034
2022-11-20 08:13:43,727 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-32, eta: 0:03:12, time: 3.233, data_time: 3.015, memory: 10569, loss: 0.4678
2022-11-20 08:14:15,807 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-32, eta: 0:02:40, time: 3.209, data_time: 2.968, memory: 10569, loss: 0.4487
2022-11-20 08:14:15,812 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.8 task/s, elapsed: 42s, ETA:     0s

2022-11-20 08:14:59,955 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 68.6567
2022-11-20 08:15:34,388 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-33, eta: 0:02:08, time: 3.423, data_time: 3.182, memory: 10569, loss: 0.4672
2022-11-20 08:16:05,906 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-33, eta: 0:01:36, time: 3.152, data_time: 2.986, memory: 10569, loss: 0.4808
2022-11-20 08:16:37,356 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-33, eta: 0:01:04, time: 3.145, data_time: 3.021, memory: 10569, loss: 0.4345
2022-11-20 08:17:08,367 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-33, eta: 0:00:32, time: 3.101, data_time: 2.976, memory: 10569, loss: 0.4870
2022-11-20 08:17:38,893 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-33, eta: 0:00:00, time: 3.054, data_time: 2.910, memory: 10569, loss: 0.4646
2022-11-20 08:17:38,897 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.8 task/s, elapsed: 42s, ETA:     0s

2022-11-20 08:18:22,647 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 69.1542


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.835853
1,BC_01_0220,0,0.862225
2,BC_01_0233,0,0.837923
3,BC_01_0258,1,0.522227
4,BC_01_0260,1,0.552688
...,...,...,...
245,BC_01_3328,0,0.626092
246,BC_01_3404,1,0.777129
247,BC_01_3418,0,0.892858
248,BC_01_3438,0,0.667886


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
 """from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

# 6.efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k

In [ ]:
# efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k
config_file = '/content/mmclassification/configs/efficientnet/efficientnet-b0_8xb32-01norm_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth'

## model 1 : 1e-4 = 0.62604828

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = checkpoint_file
cfg.resume_from = None


cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='EfficientNet',
        arch='b0',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=1280,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[127.5, 127.5, 127.5], std=[127.5, 127.5, 127.5], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='RandomResizedCrop',
        size=224,
        efficientnet_style=True,
        interpolation='bicubic'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 08:20:06,718 - mmcv - INFO - initialize EfficientNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth', 'prefix': 'backbone'}
2022-11-20 08:20:06,720 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth


Downloading: "https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth


  0%|          | 0.00/20.4M [00:00<?, ?B/s]

2022-11-20 08:20:08,626 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 08:20:08,629 - mmcv - INFO - 
backbone.layers.0.conv.weight - torch.Size([32, 3, 3, 3]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth 
 
2022-11-20 08:20:08,630 - mmcv - INFO - 
backbone.layers.0.bn.weight - torch.Size([32]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth 
 
2022-11-20 08:20:08,632 - mmcv - INFO - 
backbone.layers.0.bn.bias - torch.Size([32]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/efficientnet/efficientnet-b0_3rdparty_8xb32-aa-advprop_in1k_20220119-26434485.pth 
 
2022-11-20 08:20:08,634 - mmcv - INFO - 
backbone.layers.1.0.depthwise_conv.conv.weight - torch.Size([32,

...

2022-11-20 09:55:24,068 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 69.6517
2022-11-20 09:55:58,214 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-32, eta: 0:04:51, time: 3.414, data_time: 2.867, memory: 10569, loss: 0.5225
2022-11-20 09:56:30,128 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-32, eta: 0:04:18, time: 3.191, data_time: 2.648, memory: 10569, loss: 0.4381
2022-11-20 09:57:01,705 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-32, eta: 0:03:46, time: 3.158, data_time: 2.615, memory: 10569, loss: 0.4646
2022-11-20 09:57:33,834 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-32, eta: 0:03:14, time: 3.213, data_time: 2.669, memory: 10569, loss: 0.5006
2022-11-20 09:58:06,004 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-32, eta: 0:02:41, time: 3.217, data_time: 2.674, memory: 10569, loss: 0.4564
2022-11-20 09:58:06,008 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.8 task/s, elapsed: 42s, ETA:     0s

2022-11-20 09:58:48,266 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 69.6517
2022-11-20 09:59:22,757 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-33, eta: 0:02:09, time: 3.448, data_time: 2.902, memory: 10569, loss: 0.4308
2022-11-20 09:59:54,810 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-33, eta: 0:01:37, time: 3.205, data_time: 2.662, memory: 10569, loss: 0.4630
2022-11-20 10:00:26,803 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-33, eta: 0:01:04, time: 3.199, data_time: 2.654, memory: 10569, loss: 0.5018
2022-11-20 10:00:59,770 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-33, eta: 0:00:32, time: 3.297, data_time: 2.752, memory: 10569, loss: 0.5142
2022-11-20 10:01:31,759 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-33, eta: 0:00:00, time: 3.199, data_time: 2.658, memory: 10569, loss: 0.5049
2022-11-20 10:01:31,764 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.8 task/s, elapsed: 42s, ETA:     0s

2022-11-20 10:02:14,007 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 66.6667


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.876180
1,BC_01_0220,0,0.654769
2,BC_01_0233,0,0.790342
3,BC_01_0258,0,0.519498
4,BC_01_0260,1,0.712032
...,...,...,...
245,BC_01_3328,0,0.512574
246,BC_01_3404,1,0.706305
247,BC_01_3418,0,0.753651
248,BC_01_3438,1,0.651191


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

'from mmcls.apis.inference import inference_model as tlqkf\ntlqkf(model, img)'

# 7.densenet169_4xb256_in1k

In [ ]:
# densenet169_4xb256_in1k
config_file = r'/content/mmclassification/configs/densenet/densenet169_4xb256_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth'


## model 1 : 1e-4 = 0.6716224511

### Config Setting

In [ ]:

from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='densenet169_4xb256_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = checkpoint_file
cfg.resume_from = None

cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='DenseNet',
        arch='169',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=1664,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    d

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 10:03:59,149 - mmcv - INFO - initialize DenseNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth', 'prefix': 'backbone'}
2022-11-20 10:03:59,150 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth


Downloading: "https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth" to /root/.cache/torch/hub/checkpoints/densenet169_4xb256_in1k_20220426-a2889902.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

2022-11-20 10:04:02,770 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 10:04:02,772 - mmcv - INFO - 
backbone.stem.0.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth 
 
2022-11-20 10:04:02,774 - mmcv - INFO - 
backbone.stem.1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth 
 
2022-11-20 10:04:02,775 - mmcv - INFO - 
backbone.stem.1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb256_in1k_20220426-a2889902.pth 
 
2022-11-20 10:04:02,776 - mmcv - INFO - 
backbone.stages.0.block.0.norm1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/densenet/densenet169_4xb2

...

2022-11-20 11:40:32,871 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 70.1493
2022-11-20 11:41:09,301 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-32, eta: 0:04:54, time: 3.642, data_time: 2.404, memory: 12959, loss: 0.4461
2022-11-20 11:41:40,999 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-32, eta: 0:04:21, time: 3.170, data_time: 1.960, memory: 12959, loss: 0.4132
2022-11-20 11:42:14,288 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-32, eta: 0:03:48, time: 3.329, data_time: 2.105, memory: 12959, loss: 0.3666
2022-11-20 11:42:47,003 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-32, eta: 0:03:16, time: 3.271, data_time: 2.046, memory: 12959, loss: 0.4294
2022-11-20 11:43:18,310 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-32, eta: 0:02:43, time: 3.131, data_time: 1.924, memory: 12959, loss: 0.4151
2022-11-20 11:43:18,316 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-20 11:44:01,629 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 71.1443
2022-11-20 11:44:38,163 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-33, eta: 0:02:10, time: 3.653, data_time: 2.415, memory: 12959, loss: 0.4226
2022-11-20 11:45:09,987 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-33, eta: 0:01:38, time: 3.182, data_time: 1.974, memory: 12959, loss: 0.4020
2022-11-20 11:45:41,838 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-33, eta: 0:01:05, time: 3.185, data_time: 1.965, memory: 12959, loss: 0.4201
2022-11-20 11:46:12,529 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-33, eta: 0:00:32, time: 3.069, data_time: 1.846, memory: 12959, loss: 0.3818
2022-11-20 11:46:43,536 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-33, eta: 0:00:00, time: 3.101, data_time: 1.895, memory: 12959, loss: 0.4104
2022-11-20 11:46:43,540 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-20 11:47:26,914 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 70.6468


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

# 8.resnext50-32x4d_8xb32_in1k

In [ ]:
# resnext50-32x4d_8xb32_in1k
config_file = r'/content/mmclassification/configs/resnext/resnext50-32x4d_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth'

## model 1 : 1e-4 = 0.6679256331

### Config Setting

In [ ]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnext50-32x4d_8xb32_in1k'
VERSION = 1
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 30
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer_config = dict(grad_clip=None)
cfg.lr_config = dict(policy='step', step=1, gamma=0.1)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
cfg.log_config.interval = 10
cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = None
cfg.resume_from = '/content/drive/MyDrive/BreastCancer/mmclf/resnext50-32x4d_8xb32_in1k/1/latest.pth'



cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

/usr/local/lib/python3.7/dist-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '
/usr/local/lib/python3.7/dist-packages/mmcv/cnn/bricks/transformer.py:33: UserWarning: Fail to import ``MultiScaleDeformableAttention`` from ``mmcv.ops.multi_scale_deform_attn``, You should install ``mmcv-full`` if you need this module. 
  warnings.warn('Fail to import ``MultiScaleDeformableAttention`` from '


Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNeXt',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        groups=32,
        width_per_group=4,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),

### Train

In [ ]:
%cd /content/

/content


In [ ]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 13:03:30,488 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth', 'prefix': 'backbone'}
2022-11-20 13:03:30,489 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth
Downloading: "https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


  0%|          | 0.00/95.9M [00:00<?, ?B/s]

2022-11-20 13:03:36,250 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 13:03:36,253 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:03:36,255 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:03:36,256 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:03:36,258 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([128, 64, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/res

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.5 task/s, elapsed: 45s, ETA:     0s

...

2022-11-20 13:43:16,166 - mmcls - INFO - Epoch(val) [28][13]	accuracy_top-1: 67.6617
2022-11-20 13:43:53,561 - mmcls - INFO - Epoch [29][10/50]	lr: 1.000e-32, eta: 0:05:07, time: 3.739, data_time: 2.491, memory: 9361, loss: 0.2869
2022-11-20 13:44:26,891 - mmcls - INFO - Epoch [29][20/50]	lr: 1.000e-32, eta: 0:04:33, time: 3.333, data_time: 2.076, memory: 9361, loss: 0.3155
2022-11-20 13:44:59,826 - mmcls - INFO - Epoch [29][30/50]	lr: 1.000e-32, eta: 0:03:58, time: 3.294, data_time: 2.032, memory: 9361, loss: 0.3730
2022-11-20 13:45:32,199 - mmcls - INFO - Epoch [29][40/50]	lr: 1.000e-32, eta: 0:03:24, time: 3.237, data_time: 1.973, memory: 9361, loss: 0.3144
2022-11-20 13:46:04,470 - mmcls - INFO - Epoch [29][50/50]	lr: 1.000e-32, eta: 0:02:50, time: 3.227, data_time: 1.964, memory: 9361, loss: 0.2922
2022-11-20 13:46:04,474 - mmcls - INFO - Saving checkpoint at 29 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

2022-11-20 13:46:49,820 - mmcls - INFO - Epoch(val) [29][13]	accuracy_top-1: 70.1493
2022-11-20 13:47:27,491 - mmcls - INFO - Epoch [30][10/50]	lr: 1.000e-33, eta: 0:02:16, time: 3.767, data_time: 2.516, memory: 9361, loss: 0.2974
2022-11-20 13:47:59,918 - mmcls - INFO - Epoch [30][20/50]	lr: 1.000e-33, eta: 0:01:42, time: 3.243, data_time: 1.985, memory: 9361, loss: 0.3117
2022-11-20 13:48:32,869 - mmcls - INFO - Epoch [30][30/50]	lr: 1.000e-33, eta: 0:01:08, time: 3.295, data_time: 2.036, memory: 9361, loss: 0.3334
2022-11-20 13:49:05,477 - mmcls - INFO - Epoch [30][40/50]	lr: 1.000e-33, eta: 0:00:34, time: 3.261, data_time: 1.996, memory: 9361, loss: 0.3467
2022-11-20 13:49:37,825 - mmcls - INFO - Epoch [30][50/50]	lr: 1.000e-33, eta: 0:00:00, time: 3.235, data_time: 1.983, memory: 9361, loss: 0.3397
2022-11-20 13:49:37,829 - mmcls - INFO - Saving checkpoint at 30 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

2022-11-20 13:50:22,667 - mmcls - INFO - Epoch(val) [30][13]	accuracy_top-1: 69.1542


### Inference

In [ ]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [ ]:
model.cfg = cfg

In [ ]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [ ]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.927529
1,BC_01_0220,0,0.869187
2,BC_01_0233,0,0.952787
3,BC_01_0258,0,0.647204
4,BC_01_0260,1,0.697736
...,...,...,...
245,BC_01_3328,0,0.739380
246,BC_01_3404,0,0.530296
247,BC_01_3418,0,0.770613
248,BC_01_3438,1,0.614975


In [ ]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

## model 2

### Config Setting

In [44]:
# resnext50-32x4d_8xb32_in1k
config_file = r'/content/mmclassification/configs/resnext/resnext50-32x4d_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth'

In [45]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnext50-32x4d_8xb32_in1k'
VERSION = 2
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 300
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
#cfg.optimizer_config = dict(grad_clip=None)
cfg.optimizer_config = dict(type="GradientCumulativeOptimizerHook", cumulative_iters=4)

#cfg.lr_config = dict(policy='step', step=1, gamma=0.1)
cfg.lr_config = dict(
    policy='CosineAnnealing',
    min_lr=0,
    warmup='exp',
    warmup_iters=5,
    warmup_ratio=0.1,
    warmup_by_epoch=True)
# cfg.lr_config = dict(policy='poly', power=0.9, min_lr=1e-4, by_epoch=False)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
#cfg.log_config.interval = 10
cfg.log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')
    ])


cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = None
cfg.resume_from = '/content/drive/MyDrive/BreastCancer/mmclf/resnext50-32x4d_8xb32_in1k/1/latest.pth'



cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNeXt',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        groups=32,
        width_per_group=4,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),

### Train

In [ ]:
%cd /content/

/content


In [28]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-20 13:58:03,329 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth', 'prefix': 'backbone'}
2022-11-20 13:58:03,330 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


2022-11-20 13:58:03,549 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-20 13:58:03,551 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:58:03,552 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:58:03,554 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-20 13:58:03,554 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([128, 64, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/res

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.5 task/s, elapsed: 44s, ETA:     0s

2022-11-20 14:01:47,182 - mmcls - INFO - Epoch(val) [31][13]	accuracy_top-1: 67.1642
2022-11-20 14:02:25,871 - mmcls - INFO - Epoch [32][10/50]	lr: 9.739e-05, eta: 13:09:29, time: 3.868, data_time: 2.811, memory: 9361, loss: 0.1489
2022-11-20 14:03:00,570 - mmcls - INFO - Epoch [32][20/50]	lr: 9.739e-05, eta: 13:04:25, time: 3.384, data_time: 2.414, memory: 9361, loss: 0.0883
2022-11-20 14:03:33,367 - mmcls - INFO - Epoch [32][30/50]	lr: 9.739e-05, eta: 12:59:56, time: 3.365, data_time: 2.300, memory: 9361, loss: 0.1210
2022-11-20 14:04:09,429 - mmcls - INFO - Epoch [32][40/50]	lr: 9.739e-05, eta: 13:00:13, time: 3.522, data_time: 2.536, memory: 9361, loss: 0.1234
2022-11-20 14:04:43,905 - mmcls - INFO - Epoch [32][50/50]	lr: 9.739e-05, eta: 13:00:33, time: 3.532, data_time: 2.475, memory: 9361, loss: 0.0906
2022-11-20 14:04:43,910 - mmcls - INFO - Saving checkpoint at 32 epochs


...

2022-11-21 05:36:25,396 - mmcls - INFO - Epoch(val) [298][13]	accuracy_top-1: 74.1293
2022-11-21 05:37:01,918 - mmcls - INFO - Epoch [299][10/50]	lr: 1.097e-08, eta: 0:04:57, time: 3.567, data_time: 2.734, memory: 9361, loss: 0.0002
2022-11-21 05:37:34,997 - mmcls - INFO - Epoch [299][20/50]	lr: 1.097e-08, eta: 0:04:24, time: 3.392, data_time: 2.333, memory: 9361, loss: 0.0004
2022-11-21 05:38:06,701 - mmcls - INFO - Epoch [299][30/50]	lr: 1.097e-08, eta: 0:03:51, time: 3.084, data_time: 2.461, memory: 9361, loss: 0.0002
2022-11-21 05:38:38,680 - mmcls - INFO - Epoch [299][40/50]	lr: 1.097e-08, eta: 0:03:18, time: 3.284, data_time: 2.650, memory: 9361, loss: 0.0003
2022-11-21 05:39:11,034 - mmcls - INFO - Epoch [299][50/50]	lr: 1.097e-08, eta: 0:02:45, time: 3.156, data_time: 2.606, memory: 9361, loss: 0.0001
2022-11-21 05:39:11,040 - mmcls - INFO - Saving checkpoint at 299 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.7 task/s, elapsed: 43s, ETA:     0s

2022-11-21 05:39:55,067 - mmcls - INFO - Epoch(val) [299][13]	accuracy_top-1: 74.1293
2022-11-21 05:40:32,018 - mmcls - INFO - Epoch [300][10/50]	lr: 2.742e-09, eta: 0:02:12, time: 3.694, data_time: 2.874, memory: 9361, loss: 0.0002
2022-11-21 05:41:05,089 - mmcls - INFO - Epoch [300][20/50]	lr: 2.742e-09, eta: 0:01:39, time: 3.223, data_time: 2.322, memory: 9361, loss: 0.0001
2022-11-21 05:41:38,624 - mmcls - INFO - Epoch [300][30/50]	lr: 2.742e-09, eta: 0:01:06, time: 3.438, data_time: 2.387, memory: 9361, loss: 0.0364
2022-11-21 05:42:10,481 - mmcls - INFO - Epoch [300][40/50]	lr: 2.742e-09, eta: 0:00:33, time: 3.101, data_time: 2.130, memory: 9361, loss: 0.0003
2022-11-21 05:42:42,255 - mmcls - INFO - Epoch [300][50/50]	lr: 2.742e-09, eta: 0:00:00, time: 3.262, data_time: 2.224, memory: 9361, loss: 0.0068
2022-11-21 05:42:42,261 - mmcls - INFO - Saving checkpoint at 300 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 43s, ETA:     0s

2022-11-21 05:43:26,999 - mmcls - INFO - Epoch(val) [300][13]	accuracy_top-1: 73.6318


### Inference

In [29]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [30]:
model.cfg = cfg

In [31]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [32]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.999917
1,BC_01_0220,0,0.997958
2,BC_01_0233,0,0.990168
3,BC_01_0258,1,0.858820
4,BC_01_0260,1,0.987678
...,...,...,...
245,BC_01_3328,0,0.979271
246,BC_01_3404,0,0.689085
247,BC_01_3418,0,0.999050
248,BC_01_3438,0,0.955962


In [33]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

## model 3

### Config Setting

In [46]:
# resnext50-32x4d_8xb32_in1k
config_file = r'/content/mmclassification/configs/resnext/resnext50-32x4d_8xb32_in1k.py'
checkpoint_file = 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth'

In [47]:
from mmcv import Config
from mmcls.utils import auto_select_device
from mmcls.apis import set_random_seed

cfg = Config.fromfile(config_file)
cfg.device = auto_select_device()

######################################################
MODLE ='resnext50-32x4d_8xb32_in1k'
VERSION = 3
LEARNING_RATE = 1e-4
######################################################
BATCH_SIZE =16
SEED = 88
IMG_SIZE = 512
EPOCHS = 105
######################################################



normalize_cfg = dict(type='Normalize', mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], to_rgb=True)
# mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
cfg.data.train.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='RandomResizedCrop', size=512, scale=(0.1, 1), backend='pillow'),
    # dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='ToTensor', keys=['gt_label']),
    dict(type='Collect', keys=['img', 'gt_label'])
]
cfg.data.val.pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', size=(IMG_SIZE, IMG_SIZE), backend='pillow'),
    # dict(type='CenterCrop', crop_size=512),
    normalize_cfg,
    dict(type='ImageToTensor', keys=['img']),
    dict(type='Collect', keys=['img'])
]

cfg.data.test.pipeline = cfg.data.val.pipeline

cfg.model.head.num_classes = 2
cfg.model.head.topk = (1, )
cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file, prefix='backbone')
cfg.data.train.data_prefix = 'data/dataset/training_set/training_set'
cfg.data.train.ann_file = 'data/dataset/train.txt'
cfg.data.train.classes = 'data/dataset/classes.txt'
cfg.data.val.data_prefix = 'data/dataset/val_set/val_set'
cfg.data.val.ann_file = 'data/dataset/val.txt'
cfg.data.val.classes = 'data/dataset/classes.txt'
cfg.data.test.data_prefix = 'data/dataset/test_set/test_set'
cfg.data.test.ann_file = 'data/dataset/test.txt'
cfg.data.test.classes = 'data/dataset/classes.txt'
cfg.evaluation['metric_options']={'topk': (1, )}

# cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
#cfg.optimizer = dict(type='Adam', lr= LEARNING_RATE, eps=1e-08, weight_decay=0)
cfg.optimizer = dict(type='AdamW', lr=LEARNING_RATE, eps=1e-08, weight_decay=0)
#cfg.optimizer_config = dict(grad_clip=None)
cfg.optimizer_config = dict(type="GradientCumulativeOptimizerHook", cumulative_iters=4)

#cfg.lr_config = dict(policy='step', step=1, gamma=0.1)
cfg.lr_config = dict(
    policy='CosineAnnealing',
    min_lr=0,
    warmup='exp',
    warmup_iters=5,
    warmup_ratio=0.1,
    warmup_by_epoch=True)
# cfg.lr_config = dict(policy='poly', power=0.9, min_lr=1e-4, by_epoch=False)


cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)

cfg.work_dir = f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/{VERSION}/'
#cfg.log_config.interval = 10
cfg.log_config = dict(
    interval=10,
    hooks=[
        dict(type='TextLoggerHook'),
        dict(type='TensorboardLoggerHook')
    ])


cfg.seed = SEED
set_random_seed(SEED, deterministic=True)

cfg.data.samples_per_gpu = BATCH_SIZE # 배치사이즈
cfg.data.workers_per_gpu = 2

cfg.load_from = None
cfg.resume_from = '/content/drive/MyDrive/BreastCancer/mmclf/resnext50-32x4d_8xb32_in1k/2/epoch_100.pth'

cfg.evaluation['save_best'] = 'auto'

cfg.gpu_ids = range(1)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='ImageClassifier',
    backbone=dict(
        type='ResNeXt',
        depth=50,
        num_stages=4,
        out_indices=(3, ),
        groups=32,
        width_per_group=4,
        style='pytorch',
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth',
            prefix='backbone')),
    neck=dict(type='GlobalAveragePooling'),
    head=dict(
        type='LinearClsHead',
        num_classes=2,
        in_channels=2048,
        loss=dict(type='CrossEntropyLoss', loss_weight=1.0),
        topk=(1, )))
dataset_type = 'ImageNet'
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='RandomResizedCrop', size=224, backend='pillow'),
    dict(type='RandomFlip', flip_prob=0.5, direction='horizontal'),

### Train

In [36]:
%cd /content/

/content


In [37]:
import time
import mmcv
import os.path as osp

from mmcls.datasets import build_dataset
from mmcls.models import build_classifier
from mmcls.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
# Build the classifier
model = build_classifier(cfg.model)
model.init_weights()
# Build the dataset
datasets = [build_dataset(cfg.data.train)]
# Add `CLASSES` attributes to help visualization
model.CLASSES = datasets[0].CLASSES
# Start fine-tuning
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=True,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-11-21 05:53:08,675 - mmcv - INFO - initialize ResNeXt with init_cfg {'type': 'Pretrained', 'checkpoint': 'https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth', 'prefix': 'backbone'}
2022-11-21 05:53:08,677 - mmcv - INFO - load backbone in model from: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


load checkpoint from http path: https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth


2022-11-21 05:53:08,966 - mmcv - INFO - initialize LinearClsHead with init_cfg {'type': 'Normal', 'layer': 'Linear', 'std': 0.01}
2022-11-21 05:53:08,968 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-21 05:53:08,970 - mmcv - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-21 05:53:08,971 - mmcv - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/resnext50_32x4d_b32x8_imagenet_20210429-56066e27.pth 
 
2022-11-21 05:53:08,973 - mmcv - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([128, 64, 1, 1]): 
PretrainedInit: load from https://download.openmmlab.com/mmclassification/v0/resnext/res

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 43s, ETA:     0s

2022-11-21 05:56:48,940 - mmcls - INFO - Now best checkpoint is saved as best_accuracy_top-1_epoch_101.pth.
2022-11-21 05:56:48,941 - mmcls - INFO - Best accuracy_top-1 is 76.1194 at 101 epoch.
2022-11-21 05:56:48,942 - mmcls - INFO - Epoch(val) [101][13]	accuracy_top-1: 76.1194
2022-11-21 05:57:26,771 - mmcls - INFO - Epoch [102][10/50]	lr: 3.577e-07, eta: 0:10:53, time: 3.782, data_time: 2.735, memory: 9361, loss: 0.0004
2022-11-21 05:57:59,987 - mmcls - INFO - Epoch [102][20/50]	lr: 3.577e-07, eta: 0:10:14, time: 3.236, data_time: 2.266, memory: 9361, loss: 0.0006
2022-11-21 05:58:31,232 - mmcls - INFO - Epoch [102][30/50]	lr: 3.577e-07, eta: 0:09:35, time: 3.210, data_time: 2.146, memory: 9361, loss: 0.0009
2022-11-21 05:59:05,535 - mmcls - INFO - Epoch [102][40/50]	lr: 3.577e-07, eta: 0:09:01, time: 3.346, data_time: 2.376, memory: 9361, loss: 0.0012
2022-11-21 05:59:38,750 - mmcls - INFO - Epoch [102][50/50]	lr: 3.577e-07, eta: 0:08:27, time: 3.406, data_time: 2.349, memory: 9361

...

2022-11-21 06:03:54,373 - mmcls - INFO - Epoch(val) [103][13]	accuracy_top-1: 76.1194
2022-11-21 06:04:31,141 - mmcls - INFO - Epoch [104][10/50]	lr: 8.949e-08, eta: 0:05:04, time: 3.676, data_time: 2.630, memory: 9361, loss: 0.0019
2022-11-21 06:05:03,787 - mmcls - INFO - Epoch [104][20/50]	lr: 8.949e-08, eta: 0:04:29, time: 3.180, data_time: 2.323, memory: 9361, loss: 0.0003
2022-11-21 06:05:37,605 - mmcls - INFO - Epoch [104][30/50]	lr: 8.949e-08, eta: 0:03:56, time: 3.467, data_time: 2.787, memory: 9361, loss: 0.0007
2022-11-21 06:06:09,921 - mmcls - INFO - Epoch [104][40/50]	lr: 8.949e-08, eta: 0:03:21, time: 3.147, data_time: 2.599, memory: 9361, loss: 0.0003
2022-11-21 06:06:42,231 - mmcls - INFO - Epoch [104][50/50]	lr: 8.949e-08, eta: 0:02:48, time: 3.316, data_time: 2.696, memory: 9361, loss: 0.0013
2022-11-21 06:06:42,237 - mmcls - INFO - Saving checkpoint at 104 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

2022-11-21 06:07:27,070 - mmcls - INFO - Epoch(val) [104][13]	accuracy_top-1: 73.1343
2022-11-21 06:08:05,875 - mmcls - INFO - Epoch [105][10/50]	lr: 2.238e-08, eta: 0:02:15, time: 3.795, data_time: 2.826, memory: 9361, loss: 0.0006
2022-11-21 06:08:38,641 - mmcls - INFO - Epoch [105][20/50]	lr: 2.238e-08, eta: 0:01:41, time: 3.362, data_time: 2.302, memory: 9361, loss: 0.0003
2022-11-21 06:09:10,660 - mmcls - INFO - Epoch [105][30/50]	lr: 2.238e-08, eta: 0:01:07, time: 3.117, data_time: 2.133, memory: 9361, loss: 0.0005
2022-11-21 06:09:43,438 - mmcls - INFO - Epoch [105][40/50]	lr: 2.238e-08, eta: 0:00:33, time: 3.363, data_time: 2.297, memory: 9361, loss: 0.0008
2022-11-21 06:10:14,236 - mmcls - WARNING - Loss will be divided by 2 in the last 2 iterations because they are not enough for 4 cumulative_iters.
2022-11-21 06:10:15,416 - mmcls - WARNING - Loss will be divided by 2 in the last 2 iterations because they are not enough for 4 cumulative_iters.
2022-11-21 06:10:16,263 - mmcls 

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 201/201, 4.6 task/s, elapsed: 44s, ETA:     0s

2022-11-21 06:11:01,262 - mmcls - INFO - Epoch(val) [105][13]	accuracy_top-1: 75.6219


### Inference

In [38]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [39]:
model.cfg = cfg

In [40]:
%cd /content/
import pandas as pd
import tqdm
df_submission = pd.read_csv('./data/sample_submission.csv')

for i in df_submission.index:
  img = mmcv.imread('./data/dataset/test_set/test_set/' + df_submission.loc[i, 'ID'] + '.png')
  result = inference_model(model, img)
  df_submission.loc[i, 'N_category'] = result['pred_label']
  df_submission.loc[i, 'pred_score'] = result['pred_score']
  # df_submission.loc[i, 'pred_class'] = result['pred_class']

/content


In [41]:
# 출력
display(df_submission)

,ID,N_category,pred_score
0,BC_01_0011,1,0.999871
1,BC_01_0220,0,0.998419
2,BC_01_0233,0,0.997688
3,BC_01_0258,0,0.875928
4,BC_01_0260,1,0.719030
...,...,...,...
245,BC_01_3328,0,0.980124
246,BC_01_3404,0,0.709068
247,BC_01_3418,0,0.999031
248,BC_01_3438,0,0.984655


In [42]:
# 저장
df_submission.to_csv(f'/content/drive/MyDrive/project/BreastCancer/mmclf/{MODLE}/MMC_submit_{MODLE}_{VERSION}.csv', index=False)

In [ ]:
"""from mmcls.apis.inference import inference_model as tlqkf
tlqkf(model, img)"""

In [48]:
from mmcls.apis.inference import inference_model#, init_model, show_result_pyplot
# %matplotlib inline

In [51]:
import torch
model = torch.load('/content/drive/MyDrive/BreastCancer/mmclf/resnext50-32x4d_8xb32_in1k/3/best_accuracy_top-1_epoch_101.pth')

In [53]:
model.eval()

AttributeError: ignored